In [1]:
# Install required packages
!pip install gradio openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

# Import Libraries and Setup Environment

In [2]:
import gradio as gr
import os
import re
import json
import datetime
import uuid
import time
from openai import OpenAI  # Updated import for OpenAI v1.0.0+
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure OpenAI API with the new client approach
#client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# If you don't have .env file, uncomment and add your key directly:
client = OpenAI(api_key="your api key")

# Define Conversation States and Data Structures

In [3]:
# Conversation states
class ConversationState:
    GREETING = 0
    NAME = 1
    EMAIL = 2
    PHONE = 3
    EXPERIENCE = 4
    POSITION = 5
    LOCATION = 6
    TECH_STACK = 7
    TECHNICAL_QUESTIONS = 8
    CONCLUSION = 9

# Candidate data structure
class Candidate:
    def __init__(self):
        self.id = str(uuid.uuid4())
        self.name = ""
        self.email = ""
        self.phone = ""
        self.experience = ""
        self.position = ""
        self.location = ""
        self.tech_stack = ""
        self.interview_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.technical_responses = []

    def to_dict(self):
        return {
            "id": self.id,
            "name": self.name,
            "email": self.email,
            "phone": self.phone,
            "experience": self.experience,
            "position": self.position,
            "location": self.location,
            "tech_stack": self.tech_stack,
            "interview_date": self.interview_date,
            "technical_responses": self.technical_responses
        }

    def save_to_json(self, filename=None):
        if not filename:
            filename = f"candidate_{self.id}.json"
        try:
            with open(filename, 'w') as f:
                json.dump(self.to_dict(), f, indent=4)
            print(f"Saved candidate data to {filename}")
            return True
        except Exception as e:
            print(f"Error saving candidate data: {e}")
            return False


# Define Prompt Templates

In [4]:
# System prompt to define the chatbot's behavior
SYSTEM_PROMPT = """
You are an AI hiring assistant for TalentScout, a recruitment agency specializing in technology placements.
Your role is to gather essential information from candidates and assess their technical skills.
Be professional, courteous, and focused on the recruitment process.
Keep responses concise and relevant to the hiring context.
DO NOT ask multiple questions at once - focus on one information request at a time.
DO NOT make up information about the candidate.
"""

# Greeting prompt
GREETING_PROMPT = """
Introduce yourself as TalentScout's AI hiring assistant. Explain that you'll be collecting some
basic information and then asking a few technical questions based on their skills.
Ask for their full name to begin the process.
"""

# Tech stack prompt
TECH_STACK_PROMPT = """
Ask the candidate to list their technical skills, including programming languages, frameworks,
databases, and tools they are proficient in. Request them to be specific and comprehensive.
"""

# Technical questions prompt - Improved for better questions
TECHNICAL_QUESTIONS_PROMPT = """
Based on the candidate's tech stack: {tech_stack}, generate 3-5 relevant technical questions
that would help assess their proficiency. Follow these guidelines:
1. Questions should be specific to the exact technologies mentioned (not generic)
2. Include a mix of knowledge-based questions and problem-solving scenarios
3. Questions should be appropriate for someone with {years_experience} years of experience
4. For programming languages, include at least one algorithmic or coding challenge
5. For frameworks, include questions about best practices and common pitfalls
6. For databases, include questions about optimization and data modeling

Ask one question at a time and wait for the response before asking the next question.
Start with an easier question and gradually increase difficulty.
"""

# Conclusion prompt
CONCLUSION_PROMPT = """
Thank the candidate for their time and participation. Inform them that TalentScout will
review their information and be in touch if they're a good match for available positions.
Wish them well in their job search.
"""


# Implement Utility Functions

In [5]:
# Validation functions
def validate_email(email):
    """Validate email format"""
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return re.match(pattern, email) is not None

def validate_phone(phone):
    """Validate phone number format"""
    pattern = r'^\+?[0-9]{10,15}$'
    return re.match(pattern, phone) is not None

# Function to interact with OpenAI API with error handling - Updated for OpenAI v1.0.0+
def get_completion(messages, model="gpt-3.5-turbo", temperature=0.7, retries=3):
    """Call the LLM with error handling and retries"""
    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature
            )
            return response.choices[0].message.content
        except Exception as e:
            if attempt == retries - 1:
                # Last attempt failed, return a fallback response
                print(f"Error calling LLM API: {e}")
                return "I'm having trouble connecting right now. Please try again, or provide your response to the last question."
            # Wait before retrying (exponential backoff)
            time.sleep(2 ** attempt)

# Function to check for conversation ending keywords
def is_conversation_ending(message):
    """Check if the message contains conversation-ending keywords"""
    keywords = ["goodbye", "bye", "exit", "quit", "end"]
    return any(keyword in message.lower() for keyword in keywords)

# Function to get appropriate reprompt for a given state
def get_reprompt(state):
    """Get an appropriate reprompt for the given conversation state"""
    prompts = {
        ConversationState.NAME: "Could you please provide your full name?",
        ConversationState.EMAIL: "Please provide a valid email address where we can reach you.",
        ConversationState.PHONE: "Please provide a valid phone number where we can reach you.",
        ConversationState.EXPERIENCE: "How many years of professional experience do you have?",
        ConversationState.POSITION: "What position(s) are you interested in applying for?",
        ConversationState.LOCATION: "What is your current location?",
        ConversationState.TECH_STACK: "Please list your technical skills, including programming languages, frameworks, databases, and tools."
    }
    return prompts.get(state, "Could you please provide the information I requested?")


# Create Tech Stack Analyzer Function

In [6]:
def analyze_tech_stack(tech_stack_text):
    """Analyze and categorize the candidate's tech stack"""
    tech_analysis_prompt = f"""
    Analyze this tech stack: "{tech_stack_text}"

    1. Identify all technologies mentioned
    2. Categorize them into: Programming Languages, Frameworks, Databases, Cloud Services, Tools
    3. For each identified technology, note the appropriate experience level to ask questions about (beginner, intermediate, advanced)

    Return your analysis in a structured JSON format.
    """

    try:
        analysis_messages = [
            {"role": "system", "content": "You are a technical recruiter analyzing candidate skills."},
            {"role": "user", "content": tech_analysis_prompt}
        ]
        analysis_result = get_completion(analysis_messages)

        return analysis_result
    except Exception as e:
        print(f"Error analyzing tech stack: {e}")
        return tech_stack_text


# Implement the Core Conversation Handler

In [7]:
# Initialize global variables
conversation_state = ConversationState.GREETING
candidate = Candidate()
messages = [{"role": "system", "content": SYSTEM_PROMPT}]
question_count = 0
current_tech_question = ""
tech_stack_analysis = ""

def chat_with_bot(message, history):
    global conversation_state, candidate, messages, question_count, current_tech_question, tech_stack_analysis

    # Check for conversation-ending keywords
    if is_conversation_ending(message):
        # Save candidate data before ending
        try:
            if candidate.name and candidate.email:  # Only save if we have basic info
                candidate.save_to_json()
        except Exception as e:
            print(f"Error saving candidate data: {e}")

        # Return message in the correct format for messages type chatbot
        return [{"role": "assistant", "content": "Thank you for your time. The conversation has been ended. Feel free to refresh the page to start a new chat."}]

    # Add user message to messages history
    messages.append({"role": "user", "content": message})

    # Process based on conversation state
    if conversation_state == ConversationState.GREETING:
        if not history:  # First message
            bot_response = get_completion([
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": GREETING_PROMPT}
            ])
        else:
            candidate.name = message
            bot_response = "Thank you. Now, please provide your email address."
            conversation_state = ConversationState.EMAIL

    elif conversation_state == ConversationState.EMAIL:
        if validate_email(message):
            candidate.email = message
            bot_response = "Great! Could you share your phone number, please?"
            conversation_state = ConversationState.PHONE
        else:
            bot_response = "That doesn't look like a valid email address. Please provide a valid email in the format name@example.com."

    elif conversation_state == ConversationState.PHONE:
        if validate_phone(message):
            candidate.phone = message
            bot_response = "How many years of professional experience do you have in the technology field?"
            conversation_state = ConversationState.EXPERIENCE
        else:
            bot_response = "That doesn't look like a valid phone number. Please provide a valid phone number (10-15 digits, can start with +)."

    elif conversation_state == ConversationState.EXPERIENCE:
        candidate.experience = message
        bot_response = "What position(s) are you interested in applying for at TalentScout?"
        conversation_state = ConversationState.POSITION

    elif conversation_state == ConversationState.POSITION:
        candidate.position = message
        bot_response = "What is your current location or preferred work location?"
        conversation_state = ConversationState.LOCATION

    elif conversation_state == ConversationState.LOCATION:
        candidate.location = message
        bot_response = get_completion(messages + [
            {"role": "assistant", "content": TECH_STACK_PROMPT}
        ])
        conversation_state = ConversationState.TECH_STACK

    elif conversation_state == ConversationState.TECH_STACK:
        candidate.tech_stack = message

        # Analyze tech stack (this doesn't affect the flow but will be saved)
        tech_stack_analysis = analyze_tech_stack(message)

        # Generate the first technical question
        tech_question_prompt = TECHNICAL_QUESTIONS_PROMPT.format(
            tech_stack=candidate.tech_stack,
            years_experience=candidate.experience
        )

        # Add context about the purpose of these questions
        intro_message = "Thank you for sharing your technical background. I'll now ask you a few technical questions based on your skills to better understand your expertise level."

        tech_questions_response = get_completion(messages + [
            {"role": "user", "content": tech_question_prompt}
        ])

        bot_response = intro_message + "\n\n" + tech_questions_response
        conversation_state = ConversationState.TECHNICAL_QUESTIONS
        question_count = 1
        current_tech_question = tech_questions_response

    elif conversation_state == ConversationState.TECHNICAL_QUESTIONS:
        # Save the candidate's response to the current question
        candidate.technical_responses.append({
            "question": current_tech_question,
            "answer": message
        })

        question_count += 1

        # Check if we've asked enough questions
        if question_count > 3:  # Limiting to 3 questions
            # Move to conclusion
            conclusion_message = get_completion(messages + [
                {"role": "user", "content": CONCLUSION_PROMPT}
            ])

            bot_response = conclusion_message

            # Save candidate data
            try:
                candidate.save_to_json()
            except Exception as e:
                print(f"Error saving candidate data: {e}")

            conversation_state = ConversationState.CONCLUSION
        else:
            # Ask the next question
            next_question_prompt = f"""
            Based on the candidate's tech stack ({candidate.tech_stack}) and their experience level ({candidate.experience} years),
            ask technical question #{question_count}. Make it more challenging than the previous question.
            Tailor this question to assess deeper knowledge or problem-solving skills.
            """

            next_question = get_completion(messages + [
                {"role": "user", "content": next_question_prompt}
            ])

            bot_response = next_question
            current_tech_question = next_question

    elif conversation_state == ConversationState.CONCLUSION:
        bot_response = "Thank you again for your time. The interview is now complete. The TalentScout team will review your responses and contact you if there's a good match. Feel free to refresh the page to start a new conversation."

    # Add bot response to messages
    messages.append({"role": "assistant", "content": bot_response})

    # Return in format compatible with messages type chatbot
    return [{"role": "assistant", "content": bot_response}]


# Create Reset Function

In [8]:
# Reset function to start a new conversation
def reset_conversation():
    global conversation_state, candidate, messages, question_count, current_tech_question, tech_stack_analysis

    # Reset all state variables
    conversation_state = ConversationState.GREETING
    candidate = Candidate()
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    question_count = 0
    current_tech_question = ""
    tech_stack_analysis = ""

    # Get initial greeting
    initial_greeting = get_completion([
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": GREETING_PROMPT}
    ])

    # Return in format compatible with messages type chatbot
    return [{"role": "assistant", "content": initial_greeting}]


# Build the Gradio Interface with Experience Selection Radio Buttons

In [28]:
# Enhanced CSS for a more attractive, two-column design
custom_css = """
/* Core styles */
footer {visibility: hidden}
.gradio-container {max-width: 1100px; margin: 0 auto;}

/* Two-column layout styling */
.container {
    display: flex;
    gap: 20px;
}

.left-panel {
    background-color: Black;
    border-radius: 12px;
    padding: 15px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.1);
}

.chat-panel {
    flex-grow: 1;
    border-radius: 12px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.1);
}

/* Message styling with guaranteed visibility */
.message.user {
    background-color: black !important;
    color: white !important;
    border-radius: 12px !important;
    padding: 12px !important;
    margin: 8px 0 !important;
    max-width: 85% !important;
    margin-left: auto !important;
}

.message.bot {
    background-color: black !important;
    color: white !important;
    border-radius: 12px !important;
    padding: 12px !important;
    margin: 8px 0 !important;
    max-width: 85% !important;
    margin-right: auto !important;
}

/* Input area styling */
.input-area {
    padding: 12px;
    border-top: 1px solid #e2e8f0;
}

/* Button styling */
.primary-btn {
    background-color: #2563eb !important;
    color: white !important;
    border-radius: 8px !important;
}

.secondary-btn {
    background-color: #64748b !important;
    color: white !important;
    border-radius: 8px !important;
}

/* Radio button styling */
.radio-group .wrap {
    display: grid !important;
    grid-template-columns: 1fr 1fr;
    text-align: left;
    gap: 8px;
}

.radio-group label {
    padding: 8px !important;
    background-color: black !important;
    border: 1px solid #e2e8f0 !important;
    border-radius: 6px !important;
}

.radio-group input[type="radio"]:checked + label {
    background-color: #bfdbfe !important;
    border-color: #3b82f6 !important;
}

/* Info box styling */
.info-box {
    background-color: #27272a;
    border-left: 3px solid #2563eb;
    padding: 10px;
    margin: 10px 0;
    border-radius: 4px;
}
"""
# Experience level options for radio buttons
EXPERIENCE_OPTIONS = ["0-1 years", "1-3 years", "3-5 years", "5-10 years", "10+ years"]

# Create enhanced two-column interface
with gr.Blocks(theme=gr.themes.Base(), css=custom_css) as demo:
    gr.Markdown("# 🤖 TalentScout Hiring Assistant")

    # Two-column layout
    with gr.Row():
        # Left column for profile and controls
        with gr.Column(scale=1, elem_classes="left-panel"):
            gr.Markdown("### Candidate Profile")

            # Experience selection
            gr.Markdown("**Experience Level**")
            experience_radio = gr.Radio(
                choices=EXPERIENCE_OPTIONS,
                label="",
                info="Select your years of experience",
                elem_classes="radio-group"
            )


            # Info box with tips
            with gr.Column(elem_classes="info-box"):
                gr.Markdown("""
                ### Interview Tips
                • Be specific about your technical skills

                • Provide examples from past projects

                • Be honest about your proficiency levels

                • Ask for clarification if needed
                """)

                # Function to handle experience selection via radio buttons
        def handle_experience_selection(choice):
            global conversation_state, candidate

            candidate.experience = choice
            conversation_state = ConversationState.POSITION

            response = f"Thank you for selecting {choice} of experience. What position(s) are you interested in applying for at TalentScout?"
            messages.append({"role": "assistant", "content": response})

            return [{"role": "assistant", "content": response}]

            # Reset button
            reset_btn = gr.Button("Reset Conversation", elem_classes=["secondary-btn"])

        # Right column for chat
        with gr.Column(scale=2, elem_classes="chat-panel"):
            # Chat display
            chatbot = gr.Chatbot(
                height=500,
                value=[],
                type="messages",
                show_label=False
            )

            # Input area with send button
            with gr.Row(elem_classes="input-area"):
                msg = gr.Textbox(
                    placeholder="Type your message here...",
                    show_label=False,
                    container=False,
                    scale=5
                )
                send_btn = gr.Button("Send", elem_classes=["primary-btn"], scale=1)

    # Connect send button
    send_btn.click(
        fn=chat_with_bot,
        inputs=[msg, chatbot],
        outputs=chatbot
    ).then(
        fn=lambda: "",
        inputs=None,
        outputs=msg
    )

    # Connect Enter key
    msg.submit(
        fn=chat_with_bot,
        inputs=[msg, chatbot],
        outputs=chatbot
    ).then(
        fn=lambda: "",
        inputs=None,
        outputs=msg
    )

    # Connect experience radio
    experience_radio.change(
        fn=handle_experience_selection,
        inputs=[experience_radio, chatbot],
        outputs=chatbot
    )


     # Reset button definition
    clear = gr.Button("Reset Conversation", elem_classes=["secondary-btn"])

    # Reset button
    clear.click(
            fn=reset_conversation,
            inputs=None,
            outputs=[chatbot]
        )

    # Initialize with greeting when loaded
    demo.load(
            fn=reset_conversation,
            inputs=None,
            outputs=[chatbot]
        )

    # Set the specific initial greeting message
    initial_greeting = """Hello, I am the AI hiring assistant for TalentScout, a recruitment agency specializing in technology placements. I will be guiding you through our application process today.

To begin, could you please provide me with your full name?"""

    # Initialize with specific greeting
    def set_initial_greeting():
        return [{"role": "assistant", "content": initial_greeting}]

    demo.load(
        fn=set_initial_greeting,
        inputs=None,
        outputs=chatbot
    )




/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 1 arguments for function <function handle_experience_selection at 0x7d811f428400>, received 2.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected maximum 1 arguments for function <function handle_experience_selection at 0x7d811f428400>, received 2.
  warnings.warn(


# Launch the Application

In [29]:
if __name__ == "__main__":
    # Set initial greeting when loading
    chatbot.value = [{"role": "assistant", "content": initial_greeting}]
    demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d8daac15f153cbcd14.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
